In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
from qdrant_client import QdrantClient  # type: ignore
from qdrant_client.models import Record

qdrant = QdrantClient("http://localhost:6333")

In [6]:
# qdrant.create_collection(
#     collection_name="NY_Airbnb",
#     vectors_config=VectorParams(size=10, distance=Distance.COSINE),
# )

In [7]:
import numpy as np

np.set_printoptions(suppress=True, precision=3)

import pandas as pd

from sklearn.model_selection import train_test_split

In [8]:
data = pd.read_csv("../assets/AB_NYC_2019.csv")
print(f"Суммарное кол-во строк: {len(data)}")

data_train, data_test = train_test_split(data, test_size=0.3, random_state=0)
print(f"Train: {len(data_train)}, Test: {len(data_test)}")

Суммарное кол-во строк: 48895
Train: 34226, Test: 14669


In [9]:
from preprocess import DataPreprocessor

preprocessor = DataPreprocessor(data_train)

In [10]:
data_train_proc = preprocessor.preprocess_airbnb_data().get_preprocessed_data()

In [11]:
vectors = list(data_train_proc.to_numpy())
ids = list(data_train_proc.index)

In [12]:
qdrant.upload_points(
    collection_name="NY_Airbnb",
    points=[Record(id=id, vector=vec) for id, vec in zip(ids, vectors)],
    batch_size=64,
)